In [ ]:
# -*- coding: utf-8 -*-
"""typhoon.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1TH8ooY8N_qrzOlY11mrrf2JAlaVrF_RV
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from geopy.distance import great_circle

from sklearn import svm
from sklearn.metrics import accuracy_score
import sklearn.metrics

import torch
import torch.utils.data as Data
import datetime

import os
import random

# forecast 24-hour lead time
pre_seq = 4
batch_size = 128
epochs = 128
min_val_loss = 100
model_name = '/content/drive/My Drive/Colab Notebooks/typhoon/model_saver/Model.pkl'
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
import torch.nn as nn

from torch.nn.parameter import Parameter
from torch.autograd import Variable
import torch.nn.functional as F

import math
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/typhoon/data/CMA_train_'+str(pre_seq*6)+'h.csv', header=None)
test= pd.read_csv('/content/drive/My Drive/Colab Notebooks/typhoon/data/CMA_test_'+str(pre_seq*6)+'h.csv', header=None)

train.shape, test.shape

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x,test=0):
        x = x.unsqueeze(1)
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))

        h_out = h_out.view(-1, self.hidden_size)

        out = self.fc(h_out)

        return out

class TrainLoader(Data.Dataset):
    def __init__(self, X_wide_train, y_train):
        self.X_wide_train = X_wide_train
        self.y_train = y_train

    def __getitem__(self, index):
        return self.X_wide_train[index], self.y_train[index]

    def __len__(self):
        return len(self.X_wide_train)


CLIPER_feature =  pd.concat((train, test), axis=0)
CLIPER_feature.reset_index(drop=True, inplace=True)

X_wide_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_wide = X_wide_scaler.fit_transform(CLIPER_feature.iloc[:, 6:])
X_wide_train = X_wide[0: train.shape[0], :]

y = y_scaler.fit_transform(CLIPER_feature.loc[:, 3:4])


#****** modified


y_train = y[0: train.shape[0], :]

print("************************************************")
print(len(X_wide_train))
print(len(y_train))

#********** here is the code for extracting test code
# X_wide_test = X_wide[train.shape[0]: , :]
# y_test = y[train.shape[0]: , :]

# print(len(X_wide_test))
# print(len(y_test))



learning_rate = 0.01

input_size = 53
hidden_size = 128
num_layers = 1

num_classes = 2

model = LSTM(num_classes, input_size, hidden_size, num_layers)


if torch.cuda.is_available():
    model.cuda()
"""# Training"""

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

full_train_index = [*range(0, len(X_wide_train))]

for epoch in range(epochs):  # loop over the dataset multiple times
    starttime = datetime.datetime.now()
    train_index, val_index, _, _, = train_test_split(full_train_index, full_train_index, test_size=0.1)
    
    train_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[train_index],y_train[train_index]), 
                                                 batch_size=batch_size, shuffle=True)

    val_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size, shuffle=True)
    
      
    # training
    total_train_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_dataset):
        X_wide_train_cuda = batch_x.float()
      
        y_train_cuda = batch_y
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred_y = model(X_wide_train_cuda)
        loss = criterion(pred_y, y_train_cuda)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()

    # validation
    total_val_loss = 0
    for _, (batch_val_x, batch_val_y) in enumerate(val_dataset):

       
        X_wide_val_cuda = batch_val_x.float()
        y_val_cuda = batch_val_y

        pred_y = model(X_wide_val_cuda)
        val_loss = criterion(pred_y, y_val_cuda)
        total_val_loss += val_loss.item()

        # print statistics
    if min_val_loss > total_val_loss:
        torch.save(model.state_dict(), model_name)
        min_val_loss = total_val_loss
    endtime = datetime.datetime.now()
    print('epochs [%d/%d] cost:%.2fs train_loss: %.5f val_loss: %.5f' %
          (epoch + 1, epochs, (endtime - starttime).seconds, total_train_loss, total_val_loss))

print('Finished Training')

# here is the code for the testing

# net.load_state_dict(torch.load(model_name))
years = test[5].unique()
test_list = []

for year in years:
    temp = test[test[5] == year]
    temp = temp.reset_index(drop=True)
    test_list.append(temp)

len(test_list)

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# net = model
# net = net.to(device)
model.load_state_dict(torch.load(model_name))

#model.eval()

with torch.no_grad():
    for year, _test in zip(years, test_list):

        print(year, 'Ã¥Â¹Â´:')

        y_test_lat = _test.loc[:, 3]

        y_test_long = _test.loc[:, 4]

        X_wide_test = X_wide_scaler.transform(_test.loc[:, 6:])
        #X_wide_test = X_wide_scaler.fit|_transform(_test.loc[:, 6:])
       # X_wide_test = _test.loc[:, 6:]

        final_test_list = []



        # if torch.cuda.is_available():
        X_wide_test = Variable(torch.from_numpy(X_wide_test).float())
        #X_wide_test = X_wide_test.float().cuda()
        pred = model(X_wide_test)

        pred = y_scaler.inverse_transform(pred.cpu().detach().numpy())

        pred_lat = pred[:, 0]
        pred_long = pred[:, 1]
        true_lat = y_test_lat
        true_long = y_test_long

        diff_lat = np.abs(pred_lat - true_lat)
        diff_long = np.abs(pred_long - true_long)

        print('avg lat:', sum(diff_lat) / len(diff_lat))
        print('avg long:', sum(diff_long) / len(diff_long))

        sum_error = []
        for i in range(0, len(pred_lat)):
            sum_error.append(great_circle((pred_lat[i], pred_long[i]), (true_lat[i], true_long[i])).kilometers)

        print('avg distance error:', sum(sum_error) / len(sum_error))


: 